In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# get Max's words

nouns = '/home/jvdzwaan/data/adh/word-lists/farada-noun.txt'
verbs = '/home/jvdzwaan/data/adh/word-lists/farada-verb.txt'

with open(nouns, encoding='utf-8') as f:
    words1 = f.readlines()
with open(verbs, encoding='utf-8') as f:
    words2 = f.readlines()

farada = [w.strip() for w in words1 + words2]
len(farada)

In [ ]:
farada = set(farada)
len(farada)

In [ ]:
# taken from root-extraction-performance.ipynb
import codecs

from bs4 import BeautifulSoup

def stemmer_xml2df(fname):
    with codecs.open(fname) as f:
        soup = BeautifulSoup(f.read(), 'xml')
    
    result = []    
    for word in soup.find_all('word'):
        result.append({'word': word['value'], 'proposed_root': word.analysis['stem']})
    
    return pd.DataFrame(result)

def analyzer_xml2df(fname):
    #print(fname)
    with codecs.open(fname) as f:
        soup = BeautifulSoup(f.read(), 'xml')
    
    result = []
    
    for word in soup.find_all('word'):
        analyses = word.find_all('analysis')
        roots = [a.get('root', 'NO_ROOT') for a in analyses]
        roots = list(set(roots))
        if len(roots) == 0:
            roots.append('NOANALYSIS')
        result.append({'word': word['value'], 'proposed_root': '\\'.join(roots)})
    
    #print(len(result))
    return pd.DataFrame(result)

In [ ]:
from lxml import etree

def stemmer_xml2df2(fname):
    result = []
    
    # Extract the words
    context = etree.iterparse(fname, events=('end', ), tag=('word'))
    for event, elem in context:
        stem = None
        for a in elem.getchildren():
            if a.tag == 'analysis':
                stem = a.attrib['stem']
        result.append({'word': elem.attrib['value'], 'proposed_root': stem})
        
        # make iteration over context fast and consume less memory
        #https://www.ibm.com/developerworks/xml/library/x-hiperfparse
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    
    return pd.DataFrame(result)

In [ ]:
%%time
stemmed_file = '/home/jvdzwaan/data/tmp/adh/stemmer/0483IbnAhmadSarakhsi.Mabsut.xml'

khoja = stemmer_xml2df(stemmed_file)
print(khoja.shape)

In [ ]:
%%time
stemmed_file = '/home/jvdzwaan/data/tmp/adh/stemmer/0483IbnAhmadSarakhsi.Mabsut.xml'

khoja = stemmer_xml2df2(stemmed_file)
print(khoja.shape)

In [ ]:
khoja

In [ ]:
%%time
khoja['farada'] = khoja.apply(lambda row: row['word'] in farada, axis=1)

In [ ]:
khoja['farada'].sum()

In [ ]:
# root farada
root = 'فرض'

In [ ]:
khoja['proposed_root_farada'] = khoja[proposed_root_farada] == root

In [ ]:
khoja['proposed_root_farada'].sum()

In [ ]:
# overlap
khoja['overlap'] = (khoja['farada'] == True) & (khoja['proposed_root_farada'] == True)

In [ ]:
khoja['overlap'].sum()